## Setup

In [ ]:
import pandas as pd
from pathlib import Path
import statsmodels.formula.api as sm
import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

interim = '../../data/interim'
br = pd.read_pickle(Path(interim) / 'batting_records.pkl')
gl = pd.read_pickle(Path(interim) / 'game_logs.pkl')
events = pd.read_pickle(Path(interim) / 'events.pkl')
people = pd.read_pickle(Path(interim) / 'people.pkl')

## Merge Data

In [ ]:
## combine data
br = br.reset_index()
merged = pd.merge(br, people[['PlayerID', 'birthYear']], left_on = ['BAT_ID'], right_on=['PlayerID'], how='left')
merged = merged.set_index(['BAT_ID', 'year']).sort_values(['BAT_ID', 'year'], ascending=[True, True])

merged['Age'] = merged.index.get_level_values('year') - merged['birthYear']

merged['HPPA'] = merged['HPPA'].astype('float')
merged['H'] = merged['H'].astype('int')
merged['PA'] = merged['PA'].astype('int')
merged['Age'] = merged['Age'].astype('int')

merged = merged[~merged.HPPA.isna()]

merged = merged.sort_values(['BAT_ID', 'year'])
merged['prev_G'] = merged.groupby('BAT_ID')['G'].shift(1)

## Featurize

In [ ]:
group = merged.groupby('BAT_ID')
merged['L1_HPPA'] = group['HPPA'].shift(1)
merged['L2_HPPA'] = group['HPPA'].shift(2)
merged['L3_HPPA'] = group['HPPA'].shift(3)

merged['L1_H'] = group['H'].shift(1)
merged['L2_H'] = group['H'].shift(2)
merged['L3_H'] = group['H'].shift(3)

merged['L1_PA'] = group['PA'].shift(1)
merged['L2_PA'] = group['PA'].shift(2)
merged['L3_PA'] = group['PA'].shift(3)

## Set up Training

## Train Model

In [ ]:
x_vars = [
    'L1_HPPA', 'L2_HPPA', 'L3_HPPA', 'Age',
    'L1_H', 'L2_H', 'L3_H', 'L1_PA', 'L2_PA', 'L3_PA'
]
preprocessor =  ColumnTransformer(
    [('spot', 'passthrough', x_vars)],
    remainder='drop'
)

clf = LassoCV(
    cv=5, random_state=0, max_iter=10000, n_jobs=-1,
)

fitted_model = Pipeline([
    ('select', preprocessor),
    ('impute', IterativeImputer(random_state = 0)),
    ('poly', PolynomialFeatures(2, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', clf),
])

fitted_model.fit(merged, merged['HPPA'].astype('float'))
merged['pred_HPPA'] = fitted_model.predict(merged)

## Evaluate Results

In [ ]:
limited = merged[(merged.G >= 50) & (merged.prev_G >= 50)]

In [ ]:
result = sm.ols(formula="HPPA ~ pred_HPPA", data=limited).fit()
print(result.summary())

In [ ]:
results = cross_val_score(fitted_model, limited, limited['HPPA'].astype('float'), cv=10, n_jobs=-1, scoring='r2')
print(results)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))